In [1]:
import time
import calendar
import zmq
from data import datagenerator
from data import datahandler
from data import datastream

In [2]:
#inizializzazione variabili
#variabili relative al generatore di dati casuali utilizzato per generare i valori della candlelist
mean = 50
std = 5
delay = 1.0
#variabili relative al tempo del dato iniziale e all'intervallo temporale tra due dati adiacenti
time_year = 2015; time_month = 10; time_day = 1; time_hour = 0; time_min = 0; time_sec = 0
timestep_day = 1; timestep_hour = 0; timestep_min = 0; timestep_sec = 0
#indirizzi a cui pubblicare i dati utilizzati per effettuare il grafico e indirizzo a cui fare ping del SUB
#prima di cominciare la pubblicazione dei dati
url_pub = "tcp://*:5561"
url_ping = "tcp://localhost:5562"

In [3]:
#creazione della socket PUB utilizzata per pubblicare i dati utilizzati per effettuare il grafico seguita da ping al
#all'applicazione ricevente i dati
context = zmq.Context()
s_pub = context.socket(zmq.PUB)
s_pub.bind(url_pub)
datastream.ping(url_ping, context)

0

In [4]:
#generazione della prima candela, lista il cui 4 valori rappresentano valore minimo della giornata, 
#valore di apertura, valore di chiusura, valore massimo della giornata
candledata = sorted(datagenerator.normal_list(4, mean, std, delay))
candlelist = [candledata[0]]+datahandler.shuffle_return(candledata[1:3])+[candledata[3]]

In [5]:
#converte in millisecondi i dati riguardanti il t0 del chart e lo step temporale tra due dati
time_in_msec = 1000*int(calendar.timegm((time_year, time_month, time_day, time_hour, time_min, time_sec, 0, 0, 0)))
timestep_in_msec = (((timestep_day * 24 + timestep_hour) * 60 + timestep_min) * 60 + timestep_sec)*1000

In [ ]:
#print(candledata); print(candlelist); print(time_in_msec); print(timestep_in_msec)

In [ ]:
#in loop 
# 1) pubblica all'indirizzo fornito una tupla in cui il primo valore rappresenta l'istante temporale della misura,
#mentre il secondo valore è una candela passata come lista di quattro valori (vedi sopra)
# 2) genera la successiva candela
# 3) genera il successivo valore temporale, sommando lo step temporale prima definito all'ultimo valore temporale
while True:
    s_pub.send_pyobj((time_in_msec, candlelist))
    
    candledata = sorted(datagenerator.normal_list(4, mean, std, delay))
    candlelist = [candledata[0]]+datahandler.shuffle_return(candledata[1:3])+[candledata[3]]
    time_in_msec += timestep_in_msec
    #print(candlelist); print(time_in_msec)

In [ ]:
#candledata = sorted(datagenerator.normal_list(3, mean, std, delay))
#candlelist = [min(old_closure, candledata[0]), old_closure, candledata[1], max(old_closure, candledata[2])]

In [ ]:
#candledata

In [ ]:
#old_closure = 52.49308906399358

In [ ]:
#candlelist = [min(old_closure, candledata[0]), old_closure, candledata[1], max(old_closure, candledata[2])]

In [ ]:
#candlelist

In [ ]:
#FUNZIONE PER GENERARE UNA CANDELA CON DATO DI APERTURA PARI AL DATO DI CHIUSURA DEL GIORNO PRECEDENTE.
#PRENDE UN INGRESSO LA CANDELA DEL GIORNO PRECEDENTE E MEDIA, STD E DELAY CON CUI GENERARE I RESTANTI TRE DATI
#DEL GIORNO ATTUALE.
#MIGLIORABILE GENERANDO UNA LISTA DI DATI PER IL GIORNO ATTUALE DI CUI PRENDERE IL MINIMO, IL SECONDO PERCENTILE
#E IL MASSIMO, PER RENDERE MENO PROBABILE CHE LA CHIUSURA DEL GIORNO PRECENDENTE SIA MINORE DEL MINIMO O MAGGIORE
#DEL MASSIMO DEL GIORNO ATTUALE

#def generate_candlelist(old_candlelist, mean, std, delay = 0):
#    old_closure = old_candlelist[2]
#    candledata = sorted(datagenerator.normal_list(3, mean, std, delay))
#    candlelist = [min(old_closure, candledata[0]), old_closure, candledata[1], max(old_closure, candledata[2])]
#    return candlelist